In [1]:
import Pkg
Pkg.activate(".")
Pkg.add("Distributions")
Pkg.add("Statistics")
Pkg.add("StatsBase")
Pkg.add("ARFFFiles")
Pkg.add("DataFrames")
Pkg.add("ScikitLearn")
Pkg.add("NPZ")
Pkg.resolve();

  Activating project at `/Volumes/lodygaw/2022/learning/wdum`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `/Volumes/lodygaw/2022/learning/wdum/Project.toml`
    Updating `/Volumes/lodygaw/2022/learning/wdum/Manifest.toml`
  [7b1f6079] - FileWatching
   Resolving package versions...
  No Changes to `/Volumes/lodygaw/2022/learning/wdum/Project.toml`
  No Changes to `/Volumes/lodygaw/2022/learning/wdum/Manifest.toml`
   Resolving package versions...
  No Changes to `/Volumes/lodygaw/2022/learning/wdum/Project.toml`
  No Changes to `/Volumes/lodygaw/2022/learning/wdum/Manifest.toml`
   Resolving package versions...
  No Changes to `/Volumes/lodygaw/2022/learning/wdum/Project.toml`
  No Changes to `/Volumes/lodygaw/2022/learning/wdum/Manifest.toml`
   Resolving package versions...
  No Changes to `/Volumes/lodygaw/2022/learning/wdum/Project.toml`
  No Changes to `/Volumes/lodygaw/2022/learning/wdum/Manifest.toml`
   Resolving 

In [2]:
import DataFrames, ARFFFiles
function load_dataset(name::String, split="", dataset_directory="./datasets")
    if split == ""
        X_train, y_train = load_dataset(name, "train", dataset_directory)
        X_test, y_test = load_dataset(name, "test", dataset_directory)
        return X_train, y_train, X_test, y_test
    end
    
    @assert split in ["train", "test"] """You must specify "train" or "test" in  split parameter!"""
    @info "Loading dataset: $(name), scope: $(split)"
    
    r = Regex("$(name)Dimension[0-9]*_$(uppercase(split)).arff")
    filenames = filter(x->match(r,x)!==nothing, readdir("$(dataset_directory)/$(name)/"))
    
    X = permutedims(cat((Array{Float32}(ARFFFiles.load(DataFrames.DataFrame, "$(dataset_directory)/$(name)/$(filename)")[:,1:end-1]) for filename in filenames)..., dims=3), (1,3,2))
    y = Array(ARFFFiles.load(DataFrames.DataFrame, "$(dataset_directory)/$(name)/$(first(filenames))"))[:,end]
    
    return X,y
end

load_dataset (generic function with 3 methods)

In [3]:
X_train, y_train, X_test, y_test = load_dataset("BasicMotions");

┌ Info: Loading dataset: BasicMotions, scope: train
└ @ Main In[2]:10
┌ Info: Loading dataset: BasicMotions, scope: test
└ @ Main In[2]:10


In [4]:
include("rocket_v2.jl")

rocket = Rocket(10_000)
fit!(rocket, X_train);

# using NPZ
# npzwrite("weights.npz", rocket.kernels[1])
# npzwrite("lengths.npz", rocket.kernels[2])
# npzwrite("biases.npz", rocket.kernels[3])
# npzwrite("dilations.npz", rocket.kernels[4])
# npzwrite("paddings.npz", rocket.kernels[5])
# npzwrite("num_channel_indices.npz", rocket.kernels[6])
# npzwrite("channel_indices.npz", rocket.kernels[7])

In [5]:
X_train_transform = transform!(rocket, X_train)
X_test_transform = transform!(rocket, X_test)

40×20000 Matrix{Float32}:
 0.928571   2.63352  0.72   5.81725  …  0.8   20.8784   0.0326087  2.317
 0.607143  10.6738   0.54   8.37816     0.54  11.8453   0.347826   3.83605
 0.607143   8.78448  0.55   5.00118     0.57  10.2695   0.293478   1.62749
 0.607143  13.4096   0.59  10.3395      0.56  11.3119   0.271739   4.34426
 0.642857   7.98082  0.55   4.24509     0.5   14.7      0.304348   4.38295
 0.607143  17.3522   0.57  11.4096   …  0.52  18.1372   0.347826   4.74337
 0.607143  13.2529   0.58   5.26846     0.59   9.23376  0.217391   4.02504
 0.583333  12.8767   0.53   7.89417     0.51   8.73115  0.315217   3.68204
 0.738095  12.6367   0.56   5.55874     0.61  12.7532   0.25       4.32899
 0.583333  10.5836   0.59   6.87998     0.54  10.9058   0.282609   4.44029
 0.630952  10.335    0.53   4.75142  …  0.56   8.82414  0.358696   2.42151
 0.583333  11.2848   0.53   4.87004     0.51   8.83813  0.358696   2.96065
 0.654762   8.66403  0.53   3.71333     0.49   7.86445  0.358696   2.28285
 

In [6]:
using ScikitLearn
@sk_import linear_model : RidgeClassifierCV
alphas = [1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03]
ridge = RidgeClassifierCV(alphas=alphas, normalize=true);
ridge.fit(X_train_transform, y_train)
ridge.score(X_test_transform, y_test)

Retrieving notices: ...working... done
Solving environment: ...working... done

# All requested packages already installed.



┌ Info: Running `conda install -y -c conda-forge llvm-openmp` in root environment
└ @ Conda /Users/wiktor/.julia/packages/Conda/x2UxR/src/Conda.jl:127


LoadError: PyError ($(Expr(:escape, :(ccall(#= /Users/wiktor/.julia/packages/PyCall/ygXW2/src/pyfncall.jl:43 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <class 'ValueError'>
ValueError('Unknown label type: (array([<PyCall.jlwrap Standing>, <PyCall.jlwrap Standing>,\n       <PyCall.jlwrap Standing>, <PyCall.jlwrap Standing>,\n       <PyCall.jlwrap Standing>, <PyCall.jlwrap Standing>,\n       <PyCall.jlwrap Standing>, <PyCall.jlwrap Standing>,\n       <PyCall.jlwrap Standing>, <PyCall.jlwrap Standing>,\n       <PyCall.jlwrap Running>, <PyCall.jlwrap Running>,\n       <PyCall.jlwrap Running>, <PyCall.jlwrap Running>,\n       <PyCall.jlwrap Running>, <PyCall.jlwrap Running>,\n       <PyCall.jlwrap Running>, <PyCall.jlwrap Running>,\n       <PyCall.jlwrap Running>, <PyCall.jlwrap Running>,\n       <PyCall.jlwrap Walking>, <PyCall.jlwrap Walking>,\n       <PyCall.jlwrap Walking>, <PyCall.jlwrap Walking>,\n       <PyCall.jlwrap Walking>, <PyCall.jlwrap Walki